In [1]:
 # -*- coding: utf-8 -*-
import pandas as pd

data = pd.read_csv('TB_RECIPE_SEARCH-20231130.csv')

In [15]:
# print(data.columns)
data.head()

,RCP_SNO,RCP_TTL,CKG_NM,RGTR_ID,RGTR_NM,INQ_CNT,RCMM_CNT,SRAP_CNT,CKG_MTH_ACTO_NM,CKG_STA_ACTO_NM,CKG_MTRL_ACTO_NM,CKG_KND_ACTO_NM,CKG_IPDC,CKG_MTRL_CN,CKG_INBUN_NM,CKG_DODF_NM,CKG_TIME_NM,FIRST_REG_DT
0,128671,어묵김말이,어묵김말이,skfo0701,꽃날,10072,6,66,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내,2.007040e+13
1,128892,두부에 꼬리가 달렸어요!!,두부새우전,skfo0701,꽃날,5817,3,27,부침,일상,해물류,밑반찬,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,3인분,초급,30분이내,2.007040e+13
2,128932,입안에서 톡톡톡,알밥,skfo0701,꽃날,6975,8,36,굽기,일상,해물류,밥/죽/떡,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,2인분,초급,30분이내,2.007040e+13
3,131871,★현미호두죽,현미호두죽,cds1117,햇님&별님,3339,0,11,끓이기,일상,쌀,밥/죽/떡,현미호두죽,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,2인분,초급,30분이내,2.007040e+13
4,139247,부들부들 보들보들 북어갈비♥,북어갈비,skfo0701,꽃날,7173,3,97,굽기,술안주,건어물류,메인반찬,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,2인분,초급,60분이내,2.007050e+13


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import numpy as np

def preprocess_data(series):
    ingredient_list = []
    for data in series:
        if pd.notnull(data):  # NaN이 아닌 경우에만 처리
            ingredients = data.split("|")
            recipe_list = []
            for ingredient in ingredients:
                ingredient = ingredient.replace("[재료]", "").strip()
                parts = ingredient.split()
                if len(parts) >= 1:
                    ingredient_name = " ".join(parts[:-1])  # 재료 이름 추출
                    recipe_list.append(ingredient_name)
            if recipe_list:  # 재료가 있는 경우에만 추가
                ingredient_list.append(ingredient_name)  # 리스트를 문자열로 변환하여 추가
    
    return ingredient_list


# 데이터 전처리
recipes = preprocess_data(data.loc[:100, "CKG_MTRL_CN"])
print(recipes)

num_samples = len(recipes)
num_clusters = 10

import gensim.downloader as api

# 사전 훈련된 Word2Vec 모델
# word2vec_model = api.load('word2vec-google-news-300')

# new_data = ['peanut', 'chicken']
# embedded_data = []
# for sentence in new_data:
#     embedded_sentence = [word2vec_model[word] for word in sentence.split() if word in word2vec_model.key_to_index]
#     if embedded_sentence:
#         embedded_data.append(np.mean(embedded_sentence, axis=0))

# # print(embedded_data)

# # K-means 클러스터링 수행
# kmeans = KMeans(n_clusters=num_clusters, random_state=42)
# kmeans.fit(embedded_data)

# print(kmeans.predict(embedded_data))


if num_samples >= num_clusters:
    # 피처 벡터화
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(recipes)

    # K-means 클러스터링 수행
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(X)

    new_data = ['땅콩 닭고기 튀김가루']
    transformed_data = vectorizer.transform(new_data)
    cluster_num = kmeans.predict(transformed_data)
    print(cluster_num)

    # 클러스터링 결과 확인
    clustered_recipes = pd.DataFrame({"Recipe": recipes, "Cluster": kmeans.labels_})
    print(clustered_recipes.loc[clustered_recipes.Cluster == cluster_num, 'Recipe'])
else:   
    print("Error: Number of samples is less than number of clusters.")

['참기름', '계란', '계란 노른자', '소금', '후춧가루', '올리브유', '녹말물', '꿀', '', '', '후추가루', '', '', '', '깨', '양파', '', '달걀', '소금', '꿀', '빵가루 [양념] 백세 카레가루', '모닝빵', '', '버터', '바닐라오일', '설탕', '육수', '청or홍고추 1개 [양념] 고춧가루', '파슬리 약간 [양념] 소금', '', '후추', '후춧가루', '양겨자가루', '통깨', '오레가노', '맛소금', '레몬즙', '설탕', '유기농 초코 뮈슬리', '견과류', '플레인 요거트', '토핑용 버터', '모짜렐라 치즈 적당량 [양념] 헤비위핑크림', '참기름', '바닐라오일', '참기름', '슈거파우더', '요리당', '시칠미', '', '굴소스', '파마산치즈', '드라이이스트 8그램 (올리브유', '설탕', '후춧가루', '다진양파', '물', '', '계란물', '다진마늘', '블럭모짜렐라치즈 100그램 [양념] 설탕', '옥수수전분 적당량 [양념] 팥앙금', '가쓰오부시장국', '소금', '올리브오일', '후추가루', '치즈', '설탕', '', '소금', '유자청', '시나몬파우더 1/2ts [양념] 장식용 슈가파우더', '바닐라오일', '설탕', '견과류', '버터', '달걀 물']
[0]


ValueError: ('Lengths must match to compare', (77,), (1,))

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

def preprocess_data(series):
    ingredient_list = []
    for data in series:
        if pd.notnull(data):  # NaN이 아닌 경우에만 처리
            ingredients = data.split("|")
            recipe_list = []
            for ingredient in ingredients:
                ingredient = ingredient.replace("[재료]", "").strip()
                parts = ingredient.split()
                if len(parts) >= 1:
                    ingredient_name = " ".join(parts[:-1])  # 재료 이름 추출
                    recipe_list.append(ingredient_name)
            if recipe_list:  # 재료가 있는 경우에만 추가
                ingredient_list.append(" ".join(recipe_list))  # 리스트를 문자열로 변환하여 추가
    
    return ingredient_list


# 데이터 전처리
recipes = preprocess_data(data.loc[:100, "CKG_MTRL_CN"])
print(recipes)


num_samples = len(recipes)
num_clusters = 10
if num_samples >= num_clusters:
    # 피처 벡터화
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(recipes)
    print(X)
    
    # K-means 클러스터링 수행
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(X)

    new_data = ['땅콩 닭고기 튀김가루']
    transformed_data = vectorizer.transform(new_data)
    cluster_num = kmeans.predict(transformed_data)
    print(cluster_num)

    # 클러스터링 결과 확인
    clustered_recipes = pd.DataFrame({"Recipe": recipes, "Cluster": kmeans.labels_})
    print(clustered_recipes.loc[clustered_recipes.Cluster == cluster_num[0], 'Recipe'])
else:
    print("Error: Number of samples is less than number of clusters.")

['어묵 김밥용김 당면 양파 당근 깻잎 튀김가루 올리브유 간장 참기름', '두부 당근 고추 브로콜리 새우  계란', '밥 당근 치자단무지 신김치 무순 날치알 김가루  참기름 통깨 계란 노른자', '현미 찹쌀 호두 물 소금', '북어포 찹쌀가루 1C [양념] 간장 설탕 물 다진파 다진마늘 참기름 깨소금 후춧가루', '파스타면 [양념] 토마토 토마토 페이스트 양파 다진마늘 피망 올리브유', '건표고버섯 오이 당근 양파 사과 그외의 과일 [녹말물] 녹말가루 물 계란 노른자 1개 [탕수 소스] 물 설탕 식초 간장 녹말물', '닭 1마리 [닭양념] 통마늘 다진마늘   마늘솔트또는소금 [샐러드 드레싱] 파인애플 마요네즈 머스터드 식초1/2T(또는 꿀', '  달걀  물 ', '밥 참치캔 큰거   김밥용김 [양념]    ', '떡볶이떡 당근 파 양파 양배추 어묵    홍합살 [양념장] 고춧가루 고추장 물엿 간장 참기름 다진 마늘 깨 후추가루', '스팸 싱글 샌드위치용 토마토 상추 치즈 1장 [양념]  ', '밥 깻잎 참치 작은캔  조미 김10장 [양념] 다진   ', '새우 밥 김치 당근 감자 양파   ', '밥   밑반찬 [양념] 참기름 맛소금 깨', '밥 김치 베이컨 깻잎 양파', '식빵 스팸 오이 체다치즈(노란치즈) [양념]  ', '고구마 큰거 모차렐라치즈(피자치즈) 버터   달걀', '오징어 양파 대파 당근 청고추 홍고추 1개 [양념] 고추장 고춧가루 다진 마늘 참기름 설탕 간장 청주 후추 소금', '가래떡 호두 15알 정도 [양념] 깨 꿀', '새우 양파 김치     빵가루 [양념] 백세 카레가루', '블루베리 설탕 레몬즙 럼 모닝빵', '모닝빵 새우 상추 토마토 피클 마요네즈 [패티양념] 다진양파     ', '고구마 으깬것 생크림 설탕 버터  모짜렐라치즈 [커스타드크림] 우유 박력분 노른자 설탕 바닐라오일 버터', '국내산 제빵용 쌀가루 계란 우유 인스턴트 드라이 이스트 소금 버터 설탕 밤조림 아몬드 슬라이스 30그램. [쿠키반죽] 버터 설탕 노른자 박력분 

c:\Users\HYEONWOO\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [21]:
from transformers import GPT2Tokenizer, GPT2Model
import torch

# 데이터 전처리
ingredients = recipes[:10]
ingredient_texts = [" ".join(sublist) for sublist in ingredients]
text = " ".join(ingredient_texts)

# 모델 및 토크나이저 로드
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")

# 토크나이즈 및 토큰 인코딩
input_ids = tokenizer.encode(text, return_tensors="pt")

# 모델 호출
with torch.no_grad():
    outputs = model(input_ids)

# 출력 해석
# 여기서는 간단히 출력을 확인하는 것으로 대체합니다.
print(outputs)

# 결과 표시
# 출력 결과를 해석하여 유사한 류의 식재료를 그룹화하여 출력합니다.
# 예를 들어, 여기서는 각 그룹의 식재료들을 출력하는 것으로 대체합니다.
for ingredient_group, ingredient_text in zip(ingredients, ingredient_texts):
    print("식재료 그룹:", ingredient_group)
    print("텍스트:", ingredient_text)
    print("=" * 50)


BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-0.0490, -0.1850, -0.3257,  ..., -0.0958, -0.1116, -0.1825],
         [ 0.0617, -0.8004,  0.6519,  ..., -0.1658,  0.2724,  0.2407],
         [-0.3222, -0.8370, -0.8732,  ...,  0.1230,  0.1367,  0.5605],
         ...,
         [ 0.4944, -0.3187,  0.3258,  ...,  0.2068, -0.1860,  0.2490],
         [ 0.3921, -0.2736,  0.2728,  ...,  0.1774, -0.2009,  0.0985],
         [ 0.2798, -0.2082,  0.1031,  ...,  0.1691, -0.1055, -0.0130]]]), past_key_values=((tensor([[[[-1.6443,  2.2470,  0.8178,  ..., -1.1460, -0.6263,  1.6173],
          [-2.2578,  3.0149,  2.0628,  ...,  0.3178, -0.9223,  2.1013],
          [-1.9055,  3.5073,  2.0829,  ..., -0.6066, -2.1582,  2.5068],
          ...,
          [ 1.9951, -2.2413, -0.9001,  ..., -0.6921,  3.6162, -0.4361],
          [ 1.9528, -2.2421, -0.9206,  ..., -0.6895,  3.6630, -0.4237],
          [ 1.9287, -2.3253, -0.9176,  ..., -0.6831,  3.6660, -0.4530]],

         [[-0.0223, -0.3916, -


pip install transformers

pip install konlpy

pip install torch torchvision torchaudio 필수

JAVA가 환경변수로 등록되어 있어야 함

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from konlpy.tag import Okt

# KoBERT 모델 로드
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base")

# Konlpy 형태소 분석기 초기화
okt = Okt()

# 뉴스 기사 본문 입력
text = """
    파이썬에는 NLP 관련 도구가 정말 많다. 원하는 텍스트 분석 작업에 적절한 파이썬 라이브러리는 어떻게 찾을 수 있을까? 용도별 추천 라이브러리와 장단점을 살펴본다. 

NLP(Natural Language Processing, 자연어 처리)는 ‘음성 및 텍스트를 위한 AI’라고 말할 수 있다. 음성 명령, 음성 및 텍스트 번역, 감성 분석, 텍스트 요약, 언어와 관련된 여타 많은 애플리케이션은 그 동안 크게 개선됐다. 분석에 사용되는 NLP의 성능 또한 딥러닝을 통해 획기적으로 향상됐다.

파이썬 언어는 NLP를 포함한 모든 종류의 머신러닝에 유용한 프론트엔드를 제공한다. 하지만 파이썬 생태계에는 고를 NLP가 너무 많아 당황스러운 것도 사실이다. 여기서는 주요 파이썬용 라이브러리의 사용례와 장단점 그리고 인기도를 살펴본다. 
 
CoreNLP
스탠포드 대학(Stanford University)이 만든 CoreNLP 라이브러리는 NLP 예측 및 분석 작업을 대규모로 수행할 수 있게 해주는 실용 단계의 NLP 솔루션이다. CoreNLP는 자바(Java)로 작성됐지만, 이를 위한 API와 여러 파이썬 패키지가 등장해 있는 상태다. Stanza로 불리는 네이티브 NLP 라이브러리가 그 중 하나다.

"""

# 형태소 분석 및 품사 태깅
nouns = []
for token in okt.pos(text):
    # 명사만 추출
    if token[1] in ["Noun", "Alpha"]:
        nouns.append(token[0])

# KoBERT 모델을 이용한 핵심 단어 추출
inputs = tokenizer(" ".join(nouns), return_tensors="pt")
outputs = model(**inputs)
predictions = outputs.logits.argmax(dim=-1)

# 단어 빈도 계산
word_counts = {}
for word in nouns:
    word_counts[word] = word_counts.get(word, 0) + 1

# 결과 출력
for word, count in sorted(word_counts.items(), key=lambda x: -x[1]):
    print(f"{word}: {count}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NLP: 8
파이썬: 6
라이브러리: 5
텍스트: 4
분석: 4
수: 3
음성: 3
및: 3
CoreNLP: 3
관련: 2
작업: 2
장단점: 2
위: 2
언어: 2
그: 2
러닝: 2
로: 2
도구: 1
정말: 1
용도: 1
별: 1
추천: 1
Natural: 1
Language: 1
Processing: 1
자연어: 1
처리: 1
AI: 1
말: 1
명령: 1
번역: 1
감성: 1
요약: 1
여타: 1
애플리케이션: 1
동안: 1
크게: 1
개선: 1
사용: 1
의: 1
성능: 1
또한: 1
딥: 1
통해: 1
획기: 1
향상: 1
를: 1
포함: 1
모든: 1
종류: 1
머신: 1
프론트엔드: 1
제공: 1
생태계: 1
당황: 1
것: 1
사실: 1
여기: 1
주요: 1
용: 1
용례: 1
인기: 1
스탠포드: 1
대학: 1
Stanford: 1
University: 1
이: 1
예측: 1
대규모: 1
수행: 1
실용: 1
단계: 1
솔루션: 1
자바: 1
Java: 1
작성: 1
API: 1
여러: 1
패키지: 1
등장: 1
상태: 1
Stanza: 1
네이티: 1
브: 1
중: 1
하나: 1


In [5]:
import torch
import torchvision
import torchaudio

print(f"PyTorch 버전: {torch.__version__}")
print(f"Torchvision 버전: {torchvision.__version__}")
print(f"Torchaudio 버전: {torchaudio.__version__}")

PyTorch 버전: 2.2.2+cpu
Torchvision 버전: 0.17.2+cpu
Torchaudio 버전: 2.2.2+cpu


In [6]:
 import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# KoBERT 모델 로드
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base")

# 토큰화 및 모델 입력 준비
inputs = []
for index, row in data.iterrows():
    text = " ".join(row.dropna())  # NaN 값 제거 후 단어들을 하나의 문자열로 결합
    inputs.append(tokenizer(text, return_tensors="pt"))

# 모델에 입력하고 예측 수행
outputs = []
for input_data in inputs:
    output = model(**input_data)
    outputs.append(output)

# 결과 출력
for output in outputs:
    predictions = output.logits.argmax(dim=-1)
    print(predictions)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: sequence item 0: expected str instance, int found